In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# ml
import catboost 
from catboost import Pool, CatBoostClassifier

from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score

/kaggle/input/playground-series-s4e6/sample_submission.csv
/kaggle/input/playground-series-s4e6/train.csv
/kaggle/input/playground-series-s4e6/test.csv


In [2]:
class Config: 
    train_path=  '/kaggle/input/playground-series-s4e6/train.csv'
    test_path = '/kaggle/input/playground-series-s4e6/test.csv'

    # cat - categorical
    # nocat - not categorical
    input_columns_params = {
        "Marital status": 'cat', 
        'Application mode': 'nocat', 
        'Application order': 'nocat', 
        'Course': 'nocat', 
        'Daytime/evening attendance': 'cat', 
        'Previous qualification': 'nocat', 
        'Previous qualification (grade)': 'nocat', 
        'Nacionality': 'cat', 
        "Mother's qualification": 'nocat', 
        "Father's qualification": 'nocat', 
        "Mother's occupation": "cat", 
        "Father's occupation": 'cat', 
        'Admission grade': 'nocat',  
        'Educational special needs': 'cat', 
        'Debtor': 'cat', 
        "Tuition fees up to date": 'cat', 
        'Gender': 'cat', 
        'Scholarship holder': 'cat', 
        'Age at enrollment': 'nocat', 
        'International': 'cat', 
        'Curricular units 1st sem (credited)': 'cat',
        'Curricular units 1st sem (enrolled)': 'cat',
        'Curricular units 1st sem (evaluations)': 'nocat',
        'Curricular units 1st sem (approved)': 'cat',
        'Curricular units 1st sem (grade)': 'nocat',
        'Curricular units 1st sem (without evaluations)': 'nocat',

        'Curricular units 2nd sem (credited)': 'cat',
        'Curricular units 2nd sem (enrolled)': 'cat',
        'Curricular units 2nd sem (evaluations)': 'nocat',
        'Curricular units 2nd sem (approved)': 'cat',
        'Curricular units 2nd sem (grade)': 'nocat',
        'Curricular units 2nd sem (without evaluations)': 'nocat',
        'Unemployment rate': 'nocat', 
        'Inflation rate': 'nocat', 
        'GDP': 'nocat', 
    }
    
    output_columns_params = {"Target": 'cat'}
    
    best_params = {'n_estimators': 1989, 'learning_rate': 0.0356738607013657, 'feature_fraction': 0.5770313576324284, 'lambda_l1': 0.3486696099396136, 'lambda_l2': 2.4285720768594317, 'num_leaves': 997, 'max_depth': 6, 'colsample_bytree': 0.6290151656741123, 'min_child_samples': 33, 'min_sum_hessian_in_leaf': 4.9588242047603215, 'min_gain_to_split': 0.9988138918047753, 'max_bin': 195, 'top_rate': 0.5833584713352055}
    
    lightgbm_model_params = {
                 'objective': 'multiclass', # multiclass target: 'Graduated', 'Dropout', or 'Enrolled'
                 'data_sample_strategy': 'goss', # Gradient-based One-Sided Sampling
                 'tree_learner': 'feature', # split nodes based on the best feature
                **best_params, 
                 'verbose': -1, # Turn off warnings and model logs for a cleaner look
                 'random_state': 1,  # Random state value for repeatablity,
                'early_stopping_rounds': 2000, 
}
    
    n_splits = 40
    n_repeats = 1
    seed = 42
    
cfg= Config() 

In [3]:
df = pd.read_csv(cfg.train_path)
test_df = pd.read_csv(cfg.test_path)
df.head()

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0,1,1,1,9238,1,1,126.0,1,1,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate
1,1,1,17,1,9238,1,1,125.0,1,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout
2,2,1,17,2,9254,1,1,137.0,1,3,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout
3,3,1,1,3,9500,1,1,131.0,1,19,...,0,8,11,7,12.820000,0,11.1,0.6,2.02,Enrolled
4,4,1,1,2,9500,1,1,132.0,1,19,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate


In [4]:
def add_economic_interaction_features(df):    
    df['Economic_interaction_add'] = df['Unemployment rate'] * df['Inflation rate'] * df['GDP']
    df['Economic_interaction_add'] = df['Unemployment rate'] + df['Inflation rate'] + df['GDP']
    df['Economic_interaction_sub'] = df['Unemployment rate'] - df['Inflation rate'] - df['GDP']  
    df['Economic_interaction_div1'] = df['Unemployment rate'] / (df['Inflation rate'] + 1e-6)
    #df['Economic_interaction_div2'] = df['Inflation rate'] / (df['GDP'] + 1e-6)
    #df['Economic_interaction_div3'] = df['GDP'] / (df['Unemployment rate'] + 1e-6)   
    df['Economic_interaction_comb1'] = (df['Unemployment rate'] + df['Inflation rate']) * df['GDP']
    df['Economic_interaction_comb2'] = (df['Unemployment rate'] * df['Inflation rate']) + df['GDP']
    df['Economic_interaction_exp1'] = df['Unemployment rate'] * df['Inflation rate']
    df['Economic_interaction_exp2'] = df['GDP'] * df['Unemployment rate']
    df['Economic_interaction_exp3'] = df['GDP'] * df['Inflation rate']
    #df['Economic_interaction_log1'] = np.log1p(df['Unemployment rate']) + np.log1p(df['Inflation rate']) + np.log1p(df['GDP'])
    #df['Economic_interaction'] = (df['Unemployment rate'] * df['Inflation rate'] * df['GDP'])
    #df['Economic_interaction_sqrt1'] = np.sqrt(df['Unemployment rate']) + np.sqrt(df['Inflation rate']) + np.sqrt(df['GDP'])
   # df['Economic_interaction_sqrt2'] = np.sqrt(df['Unemployment rate'] * df['Inflation rate'] * df['GDP'])
    #df['Economic_interaction_poly1'] = df['Unemployment rate']**2 + df['Inflation rate']**2 + df['GDP']**2
    #df['Economic_interaction_poly2'] = (df['Unemployment rate'] * df['Inflation rate']) + (df['Inflation rate'] * df['GDP']) + (df['GDP'] * df['Unemployment rate'])
    return df     


def add_curriculum_interaction_features(df):
    df['Curriculum_interaction_1st_sem'] = df['Curricular units 1st sem (credited)'] * df['Curricular units 1st sem (enrolled)'] * df['Curricular units 1st sem (evaluations)'] * df['Curricular units 1st sem (approved)']
    df['Curriculum_interaction_2nd_sem'] = df['Curricular units 2nd sem (credited)'] * df['Curricular units 2nd sem (enrolled)'] * df['Curricular units 2nd sem (evaluations)'] * df['Curricular units 2nd sem (approved)']

def add_grade_interaction_features(df):
    df['Grade_interaction_1st_sem'] = df['Admission grade'] * df['Curricular units 1st sem (grade)']
    df['Grade_interaction_2nd_sem'] = df['Admission grade'] * df['Curricular units 2nd sem (grade)']

def add_qualification_interaction_features(df):
    df['Qualification_interaction'] = df['Previous qualification'] * df['Admission grade']
    df['Course_interaction'] = df['Previous qualification'] * df['Course']
    
def add_occupation_interaction_features(df):
    df["Occupation_interaction"] = df["Mother's occupation"] * df["Father's occupation"]

def add_enrollment_evaluation_interaction_features(df):
    df['Enrollment_evaluation_interaction_1st_sem'] = df['Curricular units 1st sem (enrolled)'] * df['Curricular units 1st sem (evaluations)']
    df['Enrollment_evaluation_interaction_2nd_sem'] = df['Curricular units 2nd sem (enrolled)'] * df['Curricular units 2nd sem (evaluations)']

def add_gender_marital_interaction_features(df):
    df['Gender_marital_interaction'] = df['Gender'] * df['Marital status']

def add_tuition_scholarship_interaction_features(df):
    df['Tuition_scholarship_interaction'] = df['Tuition fees up to date'] * df['Scholarship holder']  

In [5]:
def feature_engineer_train(df_train):    
    #df_train = handle_skewed_columns(df_train)   
    
    add_curriculum_interaction_features(df_train)
    add_grade_interaction_features(df_train)
#     add_age_interaction_features(df_train)
    add_economic_interaction_features(df_train)
    add_qualification_interaction_features(df_train)
    add_occupation_interaction_features(df_train)
    add_enrollment_evaluation_interaction_features(df_train)
    add_gender_marital_interaction_features(df_train)
    add_tuition_scholarship_interaction_features(df_train)
    return df_train

def prepare_X_1(df): 
    X = df[[k for k, v in cfg.input_columns_params.items()]]
    X = feature_engineer_train(X)
    # Convert real numbers for catboost 
    for i in [k for k, v in cfg.input_columns_params.items() if v == 'cat']: 
        X[i] = X[i].apply(lambda i: str(i))
        
    return X 

In [6]:
X = prepare_X_1(df)
y = df[[k for k, v in cfg.output_columns_params.items()]]

test_X = prepare_X_1(test_df)

/tmp/ipykernel_19/1485266035.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Curriculum_interaction_1st_sem'] = df['Curricular units 1st sem (credited)'] * df['Curricular units 1st sem (enrolled)'] * df['Curricular units 1st sem (evaluations)'] * df['Curricular units 1st sem (approved)']
/tmp/ipykernel_19/1485266035.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Curriculum_interaction_2nd_sem'] = df['Curricular units 2nd sem (credited)'] * df['Curricular units 2nd sem (enrolled)'] * df['C

In [7]:
# Store scores
results = []

# LGBoost

In [8]:
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm 

tqdm.pandas() 

class prepare_X_2(): 
    def __init__(self): 
        self.encoders = dict()
        
        
    def one_hot_encode_column(self, df, col_name):
        """
        One-hot encodes the specified column in the given DataFrame.

        Parameters:
        df (pd.DataFrame): The input DataFrame.
        col_name (str): The name of the column to one-hot encode.

        Returns:
        pd.DataFrame: The DataFrame with the specified column one-hot encoded.
        """
        df = df.copy()
        # Initialize new the OneHotEncoder
        encoder = OneHotEncoder(sparse_output=False, drop='if_binary', handle_unknown ='ignore')  # drop='first' to avoid multicollinearity

        # Fit and transform the specified column
        encoded_col = encoder.fit(df[[col_name]])
        self.encoders[col_name] = encoder
        
        return f"{col_name} encoder succesfully fitted"
    
    def fit(self, X): 
        for i in [k for k, v in cfg.input_columns_params.items() if v == 'cat']: 
            self.one_hot_encode_column(df = X, col_name = i)
        return X 
    
    def __call__(self, X): 
        df = X.copy()
        for col_name in [k for k, v in cfg.input_columns_params.items() if v == 'cat']:
            
            encoder = self.encoders[col_name]
            encoded_col = encoder.transform(df[[col_name]])
            # Create a DataFrame with the encoded columns
            encoded_df = pd.DataFrame(encoded_col, columns=encoder.get_feature_names_out([col_name]), index=df.index)

            # Drop the original column and concatenate the encoded columns
            df = df.drop(columns=[col_name])
            df = pd.concat([df, encoded_df], axis=1)
            
        return df 
    
    def __str__(self): 
        return ", ".join([k for k, v in self.encoders.items()])

# train Encoders 
prep2 = prepare_X_2() 
prep2.fit(pd.concat([X, test_X]))

X = prep2(X)

y_uniques = y['Target'].unique()    
y['Target'] = y['Target'].progress_apply(lambda x: [i for i, v in enumerate(y_uniques) if x == v][0])

100%|██████████| 76518/76518 [00:00<00:00, 198046.78it/s]
/tmp/ipykernel_19/928210382.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y['Target'] = y['Target'].progress_apply(lambda x: [i for i, v in enumerate(y_uniques) if x == v][0])


In [9]:
from lightgbm import LGBMClassifier 

# K-Fold Cross-Validation

kf = RepeatedStratifiedKFold(n_splits=cfg.n_splits, n_repeats=cfg.n_repeats, random_state=cfg.seed)


for train_size, test_size in kf.split(X, y): 
    train_X, train_y = X.iloc[train_size], y.iloc[train_size]
    test_X, test_y = X.iloc[test_size], y.iloc[test_size]

    clf = LGBMClassifier(**cfg.lightgbm_model_params)

    # Train the model
    clf.fit(train_X, train_y['Target'], eval_set=[(test_X, test_y['Target'])])
    
    # Make predictions
    y_pred = clf.predict(test_X)
    
    # Evaluate the model
    accuracy = accuracy_score(test_y, y_pred)
    print("Model accuracy", accuracy)
    results.append({'model': clf, 'accuracy': accuracy, 'type': 'lgbm'}) 

Model accuracy 0.8316779926816519
Model accuracy 0.8358599059069525
Model accuracy 0.8316779926816519
Model accuracy 0.8416100365917407
Model accuracy 0.8295870360690015
Model accuracy 0.832723470987977
Model accuracy 0.8243596445373759
Model accuracy 0.8452692106638787
Model accuracy 0.8264506011500261
Model accuracy 0.8384736016727653
Model accuracy 0.8306325143753267
Model accuracy 0.840041819132253
Model accuracy 0.8285415577626765
Model accuracy 0.8316779926816519
Model accuracy 0.8274960794563513
Model accuracy 0.8243596445373759
Model accuracy 0.832723470987977
Model accuracy 0.8322007318348145
Model accuracy 0.8264506011500261
Model accuracy 0.8337689492943021
Model accuracy 0.8322007318348145
Model accuracy 0.8233141662310507
Model accuracy 0.8295870360690015
Model accuracy 0.8452692106638787
Model accuracy 0.8274960794563513
Model accuracy 0.8342916884474647
Model accuracy 0.821745948771563
Model accuracy 0.836382645060115
Model accuracy 0.8274960794563513
Model accuracy 0.82

# Picking the best model

In [10]:
# Function to sort the list of dictionaries by a specified key
def sort_by_key(results, key):
    return sorted(results, key=lambda x: x[key], reverse=True)

# Sort results by "accuracy"
sorted_results = sort_by_key(results, "accuracy")

# Print sorted results
print(sorted_results[:5])
best = sorted_results[0]

[{'model': LGBMClassifier(colsample_bytree=0.6290151656741123, data_sample_strategy='goss',
               early_stopping_rounds=2000, feature_fraction=0.5770313576324284,
               lambda_l1=0.3486696099396136, lambda_l2=2.4285720768594317,
               learning_rate=0.0356738607013657, max_bin=195, max_depth=6,
               min_child_samples=33, min_gain_to_split=0.9988138918047753,
               min_sum_hessian_in_leaf=4.9588242047603215, n_estimators=1989,
               num_leaves=997, objective='multiclass', random_state=1,
               top_rate=0.5833584713352055, tree_learner='feature', verbose=-1), 'accuracy': 0.8452692106638787, 'type': 'lgbm'}, {'model': LGBMClassifier(colsample_bytree=0.6290151656741123, data_sample_strategy='goss',
               early_stopping_rounds=2000, feature_fraction=0.5770313576324284,
               lambda_l1=0.3486696099396136, lambda_l2=2.4285720768594317,
               learning_rate=0.0356738607013657, max_bin=195, max_depth=6,
   

# Making an submission

In [11]:
test_df = pd.read_csv(cfg.test_path)
test_X = prepare_X_1(test_df)

for i, best in enumerate(sorted_results[:5]): 
    if best['type'] == 'lgbm': 
        test_inputs = prep2(test_X)
        preds = best['model'].predict(test_inputs)

        preds = [y_uniques[i] for i in preds]
    elif (best['type'] == 'catboost') or (best['type'] == 'xgboost'): 
        preds = best['model'].predict(test_X)
        preds = [i[0] for i in preds]
    else: 
        print('model predict still dont writed')
    
    submission = pd.DataFrame({'id': test_df.id, 'Target': preds})
    submission.to_csv(f"submission{i}.csv", index=False)

/tmp/ipykernel_19/1485266035.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Curriculum_interaction_1st_sem'] = df['Curricular units 1st sem (credited)'] * df['Curricular units 1st sem (enrolled)'] * df['Curricular units 1st sem (evaluations)'] * df['Curricular units 1st sem (approved)']
